# Upgrade
* BVG Read
* BVG detect
* PD and Pout Regration detect
* Vbus detect

In [1]:
import logging
import sys
import pyvisa as visa
import numpy as np
import time
import matplotlib.pyplot as plt
import struct
import pandas as pd
import os
import serial
import re

from pathlib import Path

# Connect to CORA Z7

In [2]:
from JTAGDriver2 import JTAGDriver
from ToolDriver import ToolClass

bit_stream_path = './Cora_Z7_measurement_tool_7s.bit'
device_name = 'xc7z007s_1'

try:
    Dev = JTAGDriver(bit_stream_path, device_name, debug=False)

    Dev.jtag_connect()
    Dev.jtag_program()

    Tool = ToolClass(Dev)
    Tool.parse_usr_access_timestamp()
except:
    print("Please restart the Kernel and 'CORA Z7.cmd'! Then try again")

[INIT] JTAGDriver 2.0 (TCP version)
CONNECTION ESTABLISHED
Server replied: 127.0.0.1:56416,[ 23-03-13 14:07:57 ] You are connected to VIVADO server.



In [3]:
# modified to 9 bytes

def RFPGA_Power_Down():
    ## Power Down mode
    Tool.SPI_init(device = 'SPI_0', SPI_CSB_IDLE_STATE = 0, SPI_CSB_MASK = 0, SPI_CPOL = 0, SPI_CPHA = 0, SPI_SDIO_mode = 0)  
    w_data = [[[0, 0, 16, 144, 3, 195, 39, 134, 77], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0]]]
    Tool.SPI_program(device = ['SPI_0'], w_data = w_data, r_bytes = [0])
    time.sleep(1)
    
def RFPGA_Power_Config(Config):
    ## Config Power
    Tool.SPI_init(device = 'SPI_0', SPI_CSB_IDLE_STATE = 0, SPI_CSB_MASK = 0, SPI_CPOL = 0, SPI_CPHA = 0, SPI_SDIO_mode = 0)  
    w_data = [[Config, [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0]]]
    Tool.SPI_program(device = ['SPI_0'], w_data = w_data, r_bytes = [0])
    
def RFPGA_Config_Read(Config):
    ## Config Read
    Tool.SPI_init(device = 'SPI_0', SPI_CSB_IDLE_STATE = 0, SPI_CSB_MASK = 1, SPI_CPOL = 0, SPI_CPHA = 0, SPI_SDIO_mode = 0)  
    w_data = [[Config, [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0]]]
    Config_Read = Tool.SPI_program(device = ['SPI_0'], w_data = w_data, r_bytes = [9])
    return Config_Read

In [4]:
def Current(address):
    Vshunt = Tool.i2c_read(device = 'iic_0', slv_addr = address, reg_ptr = [0x01], byte_len = 2)
    I=int.from_bytes(Vshunt, "big", signed="True")*0.0125
    return I

def BGV_get():
    Tool.i2c_write(device = 'iic_0', slv_addr = 0x48, reg_ptr = [0x01], write_data = [0xE4, 0x83])
    PD = Tool.i2c_read(device = 'iic_0', slv_addr = 0x48, reg_ptr = [0x00], byte_len = 2)
    PD = int.from_bytes(PD, "big", signed="True")>>4
    FS=2.048
    PDV=PD*FS*1000/(2**11)
    print('BGV Voltage = %.3f mV'%PDV)
    return PDV

def ADC_get():
    Tool.i2c_write(device = 'iic_0', slv_addr = 0x48, reg_ptr = [0x01], write_data = [0x84, 0x83])
    time.sleep(0.5)
    PD = Tool.i2c_read(device = 'iic_0', slv_addr = 0x48, reg_ptr = [0x00], byte_len = 2)
    PDV = int.from_bytes(PD, "big", signed="True")
    FS=2.048
    PDV=round(-PDV*(FS/(2**15))*1000,2)
    return PDV

def Temp_outside(address):   
    Temp = Tool.i2c_read(device = 'iic_0', slv_addr = 0x49, reg_ptr = [0x00], byte_len = 2)
    Temp = (int.from_bytes(Temp, "big", signed="True")>>4)*0.0625
    return Temp

# RFPGA

In [5]:
from RFPGA import *

# Power Supply

In [1]:
# from instruments.PowerSupply import TP3303
# from time import sleep

# """ get all serial number """
# # TP3303.show_serial_num()

# """ connect to TP-3303 """
# power_supply = TP3303.TP3303("A9NQBHQ3A", debug=False, verbose=False, dry_run=False)

# """ turn off beep """
# power_supply.beep(on_off="OFF")

# """ swtich to independent mode """
# power_supply.mode(mode = "INDEP")

# """ set votage and  current """
# ch1_v = 6.0 # range: 0.0~32.0 V
# ch1_i = 0.3  # range: 0.0~3.2 A
# ch2_v = 6.0 # range: 0.0~32.0 V
# ch2_i = 0.5  # range: 0.0~3.2 A
# power_supply.set_config_INDEP(vset1 = ch1_v, iset1 = ch1_i, vset2 = ch2_v, iset2 = ch2_i)

# """ get votage and  current setting """
# ch1_v_setting, ch1_i_setting, ch2_v_setting, ch2_i_setting = power_supply.get_config()
# print(f"channel 1 votage setting:  {ch1_v_setting}")
# print(f"channel 1 current setting: {ch1_i_setting}")
# print(f"channel 2 votage setting:  {ch2_v_setting}")
# print(f"channel 2 current setting: {ch2_i_setting}")

# """ deactive output """
# power_supply.output(on_off = "ON")
# sleep(2)

# """ get the present output voltage and current value """
# ch1_v_out = power_supply.meas_voltage(channel = 1)
# ch1_i_out = power_supply.meas_current(channel = 1)
# ch2_v_out = power_supply.meas_voltage(channel = 2)
# ch2_i_out = power_supply.meas_current(channel = 2)
# print(f"channel 1 votage output:  {ch1_v_out}")
# print(f"channel 1 current output: {ch1_i_out}")
# print(f"channel 2 votage output:  {ch2_v_out}")
# print(f"channel 2 current output: {ch2_i_out}")

# """ deactive output """
# # power_supply.output(on_off = "OFF")


{'COM1': None, 'COM5': '066EFF494849887767255537', 'COM3': None}


In [7]:
# FIND DEVICES
POWER_SUPPLY_ID = "USB VID:PID=0403:6001 SER=A9ATTRN7A"
FPGA_ID = "USB VID:PID=10C4:EA70 SER=7 LOCATION=1-7.3:x.1"

######################################################

import serial
import re
import serial.tools.list_ports
ports = serial.tools.list_ports.comports()

for port,desc,hwid in ports:
    print(f"{port},{desc},{hwid}")
    if(hwid == POWER_SUPPLY_ID):
        print(f"[STATUS] POWER SUPPLY FOUND @ {port}")
        POWER_SUPPLY_PORT = port
    elif(hwid == FPGA_ID):
        print(f"[STATUS] FPGA FOUND @ {port}")
        FPGA_PORT = port

COM10,USB Serial Port (COM10),USB VID:PID=0403:6010 SER=210370B7C1DBB
COM1,通訊連接埠 (COM1),ACPI\PNP0501\2
COM9,USB Serial Port (COM9),USB VID:PID=0403:6001 SER=A9ATTRN7A
[STATUS] POWER SUPPLY FOUND @ COM9
COM3,Intel(R) Active Management Technology - SOL (COM3),PCI\VEN_8086&DEV_7AEB&SUBSYS_0AC11028&REV_11\3&11583659&1&B3


In [8]:
try:
    TP3303 = serial.Serial(port=POWER_SUPPLY_PORT,
                                baudrate=9600,
                                bytesize=8,
                                timeout=0.5,
                                stopbits=serial.STOPBITS_ONE,
                                write_timeout=0.5,
                                rtscts=False)
    if TP3303.isOpen(): # try to open port, if possible print message and proceed with 'while True:'
        print ("[STATUS] port is opened!")

except IOError: # if port is already opened, close it and open it again and print message
    TP3303.close()
    TP3303.open()
    print ("[STATUS] port was already open, was closed and opened again!")
        
        
# CHECK CONNECTION
TP3303.write(b'*IDN?\n')
result = TP3303.readline()
if(result == b'SN:V3.53;\r'):
    print("[STATUS] POWER SUPPLY Connection Success")
else:
    print("[Error] Please check POWER SUPPLY ID and connection!")

[STATUS] port is opened!
[STATUS] POWER SUPPLY Connection Success


In [9]:
# YOUR POWER SUPPLY SETTINGS HERE
vset1 = 2.5
iset1 = 0.3
vset2 = 3.3
iset2 = 0.1

# write
TP3303.write(str.encode("VSET1:"+ str(vset1) +"\r\n"))
TP3303.write(str.encode("ISET1:"+ str(iset1) +"\r\n"))
TP3303.write(str.encode("VSET2:"+ str(vset2) +"\r\n"))
TP3303.write(str.encode("ISET2:"+ str(iset2) +"\r\n"))
# query
TP3303.write(b'VSET1?\r\n')
vset1 = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
TP3303.write(b'ISET1?\r\n')
iset1 = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
TP3303.write(b'VSET2?\r\n')
vset2 = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
TP3303.write(b'ISET2?\r\n')
iset2 = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
print(f"Vset1 = {vset1} V, Iset1 = {iset1} A")
print(f"Vset2 = {vset2} V, Iset2 = {iset2} A")

Vset1 = 2.5 V, Iset1 = 0.3 A
Vset2 = 3.3 V, Iset2 = 0.1 A


In [10]:
def meas_supply1():
    TP3303.write(b'VOUT1?\r\n')
    vout = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
    TP3303.write(b'IOUT1?\r\n')
    iout = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
    #print(f"vout1 = {vout} V, iout1 = {iout} A")
    return(vout,iout)

def meas_supply2():
    TP3303.write(b'VOUT2?\r\n')
    vout = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
    TP3303.write(b'IOUT2?\r\n')
    iout = float((re.findall("\d+\.\d+",TP3303.readline().decode("utf-8")))[0])
    #print(f"vout2 = {vout} V, iout2 = {iout} A")
    return(vout,iout)

def POWER_SUPPLY_ON():
    TP3303.write(b'OUT1\r\n')
    
def POWER_SUPPLY_OFF():
    TP3303.write(b'OUT0\r\n')

In [16]:
POWER_SUPPLY_ON()

In [12]:
VbusI2V5 = Tool.i2c_read(device = 'iic_0', slv_addr = 0x40, reg_ptr = [0x02], byte_len = 2)
VbusI2V5 = int.from_bytes(VbusI2V5, "big")*1.25
VbusI3V3 = Tool.i2c_read(device = 'iic_0', slv_addr = 0x44, reg_ptr = [0x02], byte_len = 2)
VbusI3V3 = int.from_bytes(VbusI3V3, "big")*1.25
print(VbusI2V5, VbusI3V3)

if (VbusI2V5 < 2400 or VbusI3V3 < 3200):
    raise ValueError("Power supply error.")
else:
    print("Power supply config correct.")

mask not matched
mask not matched
mask not matched
mask not matched
0.0 0.0


ValueError: Power supply error.

In [15]:
POWER_SUPPLY_OFF()

# P5026A

<div class="alert alert-block alert-danger">
<b>Caution:</b> If NA has been calibrated, run this code rather than run the two following code block. 
</div>

>from P5026A import * <br>
>VNA = Connection() <br>
>MeasurementForm() <br>

In [17]:
from P5026A import *
VNA = Connection(dryrun = True)
VNA.write("SYST:PRES")
MeasurementForm()

In [14]:
Calibration()


[command]0.0
Ready and Press Enter! 
When the cal is over, the screen will be black.
Please wait until Cal is over and then press enter.

[command]0.0
Ready and Press Enter! 
When the cal is over, the screen will show four windows.
Please wait until Cal is over and then press enter.

[command]0.0
Ready and Press Enter! 
When the cal is over, the screen will show four windows.
Please wait until Cal is over and then press enter.
VNA has been calibrated!


# Measurement


In [18]:

def float_to_bin(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

def TF_LIST_GEN(Boolean):
    num = np.size(Boolean)
    remainder = 8 - (num % 8)
    array = np.append(Boolean, np.zeros(remainder))
    array = array.astype(int)
    bin_str = "".join(map(str, array))
    decimal_list = DECIMAL_LIST_GEN(bin_str)
    return(decimal_list)

def Regression(Pout,PD):
    Pout_w = 10 ** (Pout/10)
    z = np.polyfit(PD, Pout_w, 2)
    a = DECIMAL_LIST_GEN(float_to_bin(z[0]))
    b = DECIMAL_LIST_GEN(float_to_bin(z[1]))
    c = DECIMAL_LIST_GEN(float_to_bin(z[2]))
    # Tool.i2c_write(device = 'iic_0', slv_addr = 0x50, reg_ptr = [0x010], write_data = a+b+c)
    return z

def check():
    print("============================= POWER_SUPPLY_ON() =================================")
    # EEPROM = Tool.i2c_read(device = 'iic_0', slv_addr = 0x50, reg_ptr = [0x00], byte_len = 2)
    # uuid = "{0:02X}{1:02X}".format(EEPROM[0], EEPROM[1])
    uuid = "6060"  # fake data
    print('uuid = ',uuid)
    path = "./Measurement_Data/PGA_"+uuid

    ## Check [STATUS]
    time.sleep(0.5)
    RFPGA_Power_Down()
    
    # initialization
    RFPGA = RFPGADriver(debug=False)
    
    # check mode configuration
    check_mode_config = {
        "PA_MNCOMP": True,
        "PA_EN": True, "PA_DSVCG": 2.3, "PA_DSVG": 0.375, # PA_DSVCG, PA_DSVG unit = V
        "PD_EN": False, "PA_PSVCG": 1.9, "PA_PSVG": 0.375, # PA_PSVCG, PA_PSVG unit = V
        "PD_VDDEN": False, "PD_OUTEN": False, "BIAS_VDDEN": False, "PD_GAIN": 0b01010,
        "TEMP_OUTEN": False, "TEMP_GAIN": 6,
        "PD_RFEN": False, "COUPLER_RFEN": False, "COUPLER_50EN": True,
        
        "COMPENS_VDDEN": False, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0, "COMPENS_PS": 0,
        "TEMP_VDDEN": False,
        "TEST_CTRL": "High-Z"        
    }
    
    x = RFPGA.Gen_Bistream(config=check_mode_config, MSB=True, to_hex=False)
    
    # =======================================================================================
    
    decimal_list = DECIMAL_LIST_GEN(x)
    RFPGA_Power_Config(decimal_list)
    check = RFPGA_Config_Read(decimal_list)[0]
    RFPGA_Power_Config(decimal_list)
    time.sleep(0.5)
    Power_fixed(-5)
    time.sleep(1)
    try:
        if( list(check[:,0]) == decimal_list):
            print("[STATUS] Config Right") 
        else:
            print("[STATUS] Config Error")
            print("============================= POWER_SUPPLY_OFF() =================================")
            sys.exit(0)
        I2V5 = meas_supply1()[1]
        I3V3 = meas_supply2()[1]
        BGV = BGV_get()
        # if( I3V3 >= 0.02 or I3V3 < 0.005 or I2V5 > 0.29):
        #     print("[STATUS] Power Supply Error")
        #     print('I2V5 = %.3f A' %I2V5,'I3V3 = %.3f A' %I3V3)
        #     print("============================= POWER_SUPPLY_OFF() =================================")
        #     sys.exit(0)
        # else:
        #     print("[STATUS] Power Supply Right")
        #     print('I2V5 = %.3f A' %I2V5,'I3V3 = %.3f A' %I3V3)
        #     if(BGV<1050):
        #         raise ValueError("Band Gap Voltage too low.")
        print('I2V5 = %.3f A' %I2V5,'I3V3 = %.3f A' %I3V3)
        
        with open(path+"/BGV.txt", 'w') as file:
            file.write("str(BGV)")
    finally:
        Power_unfixed()
        RFPGA_Power_Down()
        time.sleep(0.5)
        print("============================= POWER_SUPPLY_OFF() =================================")
    return uuid
    
def Temp_Measurement():
    Start_Time = time.time()
    print('TEMP Measurement Start.')
    print("============================= POWER_SUPPLY_ON() =================================")

    RFPGA_Power_Down()
            
    # initialization
    RFPGA = RFPGADriver(debug=False)

    # temp measurement configuration
    temp_Measurement_config = {      
      "PA_MNCOMP": True,
      "PA_EN": True, "PA_DSVCG": 1.9, "PA_DSVG": 0.275, # PA_DSVCG, PA_DSVG unit = V
      "PD_EN": True, "PA_PSVCG": 1.9, "PA_PSVG": 0.300, # PA_PSVCG, PA_PSVG unit = V
      "PD_VDDEN": False, "PD_OUTEN": False, "BIAS_VDDEN": True, "PD_GAIN": 0b01010,
      "TEMP_OUTEN": True, "TEMP_GAIN": 6,
      "PD_RFEN": True, "COUPLER_RFEN": False, "COUPLER_50EN": True,

      "COMPENS_VDDEN": True, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0.040625, "COMPENS_PS": 0.040625,
      "TEMP_VDDEN": False,
      "TEST_CTRL": "High-Z"
    }
    
    x = RFPGA.Gen_Bistream(config=temp_Measurement_config, MSB=True, to_hex=False)    
    
    decimal_list = DECIMAL_LIST_GEN(x)
    RFPGA_Power_Config(decimal_list)
    time.sleep(0.5)
    Power_fixed(-5)
    time.sleep(2)

    ADC = [ADC_get()]
    Tempout = [Temp_outside(0x49)]

    df = pd.DataFrame()
    df['TEMP_in'] = ADC
    df['TEMP_out'] = Tempout
    # Save file
    if not os.path.isdir(path):
           os.makedirs(path)

    savefile_filename = path+'\TEMP_Test.csv'    
    df.to_csv(savefile_filename)
    Power_unfixed()
    End_Time = time.time()
    print('TEMP Measurement Time Elapsed: ' + str(round(End_Time-Start_Time, 2)) + ' seconds')
    print('TEMP Measurement Finish.\n')
    
def Power_Sweep_Measurement(config_num=31, Min=-20, Max=0, First=-2, Second=-5, Third=-10):
    Start_Time = time.time()
    print('Power Sweep Measurement Measurement Start.')

    # 先建立基本的參數表
    constant_map = {
                      "PA_MNCOMP": True,
                      "PA_EN": True, "PA_DSVCG": 1.9, "PA_DSVG": 0.375, # PA_DSVCG, PA_DSVG unit = V
                      "PD_EN": True, "PA_PSVCG": 1.9, "PA_PSVG": 0.375, # PA_PSVCG, PA_PSVG unit = V
                      "PD_VDDEN": False, "PD_OUTEN": False, "BIAS_VDDEN": True, "PD_GAIN": 0b01010,
                      "TEMP_OUTEN": False, "TEMP_GAIN": 6,
                      "PD_RFEN": False, "COUPLER_RFEN": False, "COUPLER_50EN": True,

                      "COMPENS_VDDEN": True, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0.040625, "COMPENS_PS": 0.040625,
                      "TEMP_VDDEN": False,
                      "TEST_CTRL": "High-Z"      
    }

    # 0~30 case 的變因
    Config_map = {
                #   0:[1.9, 0.375, 1.9, 0.375, 0, True, True, False],
                #   1:[1.9, 0.350, 1.9, 0.375, 0, True, True, False],
                #   2:[1.9, 0.325, 1.9, 0.375, 0, True, True, False],
                #   3:[1.9, 0.300, 1.9, 0.375, 0, True, True, False],
                #   4:[1.9, 0.275, 1.9, 0.375, 0, True, True, False],
                #   5:[1.9, 0.250, 1.9, 0.375, 0, True, True, False],
                  0:{"PA_DSVG": 0.375, "PA_PSVG": 0.375},                  
                  1:{"PA_DSVG": 0.350, "PA_PSVG": 0.375},
                  2:{"PA_DSVG": 0.325, "PA_PSVG": 0.375},
                  3:{"PA_DSVG": 0.300, "PA_PSVG": 0.375},
                  4:{"PA_DSVG": 0.275, "PA_PSVG": 0.375},
                  5:{"PA_DSVG": 0.250, "PA_PSVG": 0.375},
                  
                #   6:[1.9, 0.375, 1.9, 0.350, 0, True, True, False],
                #   7:[1.9, 0.350, 1.9, 0.350, 0, True, True, False],
                #   8:[1.9, 0.325, 1.9, 0.350, 0, True, True, False],
                #   9:[1.9, 0.300, 1.9, 0.350, 0, True, True, False],
                #   10:[1.9, 0.275, 1.9, 0.350, 0, True, True, False],
                #   11:[1.9, 0.250, 1.9, 0.350, 0, True, True, False],
                  6:{"PA_DSVG": 0.375, "PA_PSVG": 0.350},
                  7:{"PA_DSVG": 0.350, "PA_PSVG": 0.350},
                  8:{"PA_DSVG": 0.325, "PA_PSVG": 0.350},
                  9:{"PA_DSVG": 0.300, "PA_PSVG": 0.350},
                  10:{"PA_DSVG": 0.275, "PA_PSVG": 0.350},
                  11:{"PA_DSVG": 0.250, "PA_PSVG": 0.350},
                  
                #   12:[1.9, 0.375, 1.9, 0.325, 0, True, True, False],
                #   13:[1.9, 0.350, 1.9, 0.325, 0, True, True, False],
                #   14:[1.9, 0.325, 1.9, 0.325, 0, True, True, False],
                #   15:[1.9, 0.300, 1.9, 0.325, 0, True, True, False],
                #   16:[1.9, 0.275, 1.9, 0.325, 0, True, True, False],
                #   17:[1.9, 0.250, 1.9, 0.325, 0, True, True, False],
                  12:{"PA_DSVG": 0.375, "PA_PSVG": 0.325},
                  13:{"PA_DSVG": 0.350, "PA_PSVG": 0.325},
                  14:{"PA_DSVG": 0.325, "PA_PSVG": 0.325},
                  15:{"PA_DSVG": 0.300, "PA_PSVG": 0.325},
                  16:{"PA_DSVG": 0.275, "PA_PSVG": 0.325},
                  17:{"PA_DSVG": 0.250, "PA_PSVG": 0.325},
                                    
                #   18:[1.9, 0.375, 1.9, 0.300, 0, True, True, False],
                #   19:[1.9, 0.350, 1.9, 0.300, 0, True, True, False],
                #   20:[1.9, 0.325, 1.9, 0.300, 0, True, True, False],
                #   21:[1.9, 0.300, 1.9, 0.300, 0, True, True, False],
                #   22:[1.9, 0.275, 1.9, 0.300, 0, True, True, False],
                #   23:[1.9, 0.250, 1.9, 0.300, 0, True, True, False],                  
                  18:{"PA_DSVG": 0.375, "PA_PSVG": 0.300},
                  19:{"PA_DSVG": 0.350, "PA_PSVG": 0.300},
                  20:{"PA_DSVG": 0.325, "PA_PSVG": 0.300},
                  21:{"PA_DSVG": 0.300, "PA_PSVG": 0.300},
                  22:{"PA_DSVG": 0.275, "PA_PSVG": 0.300},
                  23:{"PA_DSVG": 0.250, "PA_PSVG": 0.300},
                  
                #   24:[1.9, 0.375, 1.9, 0.275, 0, True, True, False],
                #   25:[1.9, 0.350, 1.9, 0.275, 0, True, True, False],
                #   26:[1.9, 0.325, 1.9, 0.275, 0, True, True, False],
                #   27:[1.9, 0.300, 1.9, 0.275, 0, True, True, False],
                #   28:[1.9, 0.275, 1.9, 0.275, 0, True, True, False],
                #   29:[1.9, 0.250, 1.9, 0.275, 0, True, True, False],
                  24:{"PA_DSVG": 0.375, "PA_PSVG": 0.275},
                  25:{"PA_DSVG": 0.350, "PA_PSVG": 0.275},
                  26:{"PA_DSVG": 0.325, "PA_PSVG": 0.275},
                  27:{"PA_DSVG": 0.300, "PA_PSVG": 0.275},
                  28:{"PA_DSVG": 0.275, "PA_PSVG": 0.275},
                  29:{"PA_DSVG": 0.250, "PA_PSVG": 0.275},
                                  
                #   30:[1.9, 0.375, 1.9, 0.375, 0, True, False, False],
                  30:{
                        "PA_MNCOMP": True,
                        "PA_EN": False, "PA_DSVCG": 1.9, "PA_DSVG": 0.375, # PA_DSVCG, PA_DSVG unit = V
                        "PD_EN": False, "PA_PSVCG": 1.9, "PA_PSVG": 0.375, # PA_PSVCG, PA_PSVG unit = V
                        "PD_VDDEN": False, "PD_OUTEN": False, "BIAS_VDDEN": False, "PD_GAIN": 0b01010,
                        "TEMP_OUTEN": False, "TEMP_GAIN": 6,
                        "PD_RFEN": False, "COUPLER_RFEN": False, "COUPLER_50EN": True,

                        "COMPENS_VDDEN": True, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0.040625, "COMPENS_PS": 0.040625,
                        "TEMP_VDDEN": False,
                        "TEST_CTRL": "High-Z"
                    },
                 }

    Config_num = range(config_num)
    
    Config = np.zeros(np.size(Config_num))
    I2V5_First = np.zeros(np.size(Config_num))
    I3V3_First = np.zeros(np.size(Config_num))
    Gain_First = np.zeros(np.size(Config_num))
    I2V5_Second = np.zeros(np.size(Config_num))
    I3V3_Second = np.zeros(np.size(Config_num))
    Gain_Second = np.zeros(np.size(Config_num))
    Pout_Second = np.zeros(np.size(Config_num))
    I2V5_Third = np.zeros(np.size(Config_num))
    I3V3_Third = np.zeros(np.size(Config_num))
    Gain_Third = np.zeros(np.size(Config_num))
    Cur_Boolean = np.zeros(np.size(Config_num))

    IP1dB = np.zeros(np.size(Config_num))
    OP1dB = np.zeros(np.size(Config_num))

    print("============================= POWER_SUPPLY_ON() =================================")
    time.sleep(0.5)
    RFPGA_Power_Down()
    
    RFPGA = RFPGADriver(debug=False)
    
    # 從 case0 掃到 case30
    for i in Config_num:
        Config[i] = i
        constant_map.update(Config_map[i])  # 把變因的部分update到原本的參數表
        x = RFPGA.Gen_Bistream(config=constant_map, MSB=True, to_hex=False)
        
        decimal_list = DECIMAL_LIST_GEN(x)
        #print("deciaml_list: ", decimal_list)
        RFPGA_Power_Config(decimal_list)
        time.sleep(0.5)

        Power_sweep(Min,Max,100,1E3)
        # Auto_Scale()
        CSV_get_Power_sweep(i,path)
        time.sleep(0.5)
        IP1dB[i] = P1dB_get()[0]
        OP1dB[i] = P1dB_get()[1]

        Power_fixed(First)
        time.sleep(0.5)
        I2V5_First[i] = Current(0x40)
        I3V3_First[i] = Current(0x44)
        Gain_First[i] = Marker_get(1,2,1)[0]-Marker_get(1,1,1)[0]
        Power_unfixed()
        Power_fixed(Second)
        time.sleep(0.5)
        I2V5_Second[i] = Current(0x40)
        I3V3_Second[i] = Current(0x44)
        Gain_Second[i] = Marker_get(1,2,1)[0]-Marker_get(1,1,1)[0]
        Pout_Second[i] = Marker_get(1,2,1)[0]
        Power_unfixed()
        if I2V5_Second[i] > 160:
            Cur_Boolean[i] = 0
        elif I2V5_Second[i] <= 160:
            Cur_Boolean[i] = 1

        Power_fixed(Third)
        time.sleep(0.5)
        I2V5_Third[i] = Current(0x40)
        I3V3_Third[i] = Current(0x44)
        Gain_Third[i] = Marker_get(1,2,1)[0]-Marker_get(1,1,1)[0]
        Power_unfixed()

    df = pd.DataFrame()
    df['Config'] = Config
    df['I3V3_'+str(Third)+'dBm (mA)'] = I3V3_Third
    df['I3V3_'+str(Second)+'dBm (mA)'] = I3V3_Second
    df['I3V3_'+str(First)+'dBm (mA)'] = I3V3_First
    df['I2V5_'+str(Third)+'dBm (mA)'] = I2V5_Third
    df['I2V5_'+str(Second)+'dBm (mA)'] = I2V5_Second
    df['I2V5_'+str(First)+'dBm (mA)'] = I2V5_First
    df['Gain_'+str(Third)+'dBm (dB)'] = Gain_Third             
    df['Gain_'+str(Second)+'dBm (dB)'] = Gain_Second
    df['Gain_'+str(First)+'dBm (dB)'] = Gain_First
    df['Pout_'+str(Second)+'dBm (dBm)'] = Pout_Second
    df['IP1dB(dBm)'] = IP1dB
    df['OP1dB(dBm)'] = OP1dB
    df['Current_Compliant'] = Cur_Boolean

    if not os.path.isdir(path+'\Power_Sweep'):
           os.makedirs(path+'\Power_Sweep')

    savefile_filename = path+'\Power_Sweep\CUR、P1dB.csv'    
    df.to_csv(savefile_filename)
    
    CUR_LIST = TF_LIST_GEN(Cur_Boolean)
    # Tool.i2c_write(device = 'iic_0', slv_addr = 0x50, reg_ptr = [0x02], write_data = CUR_LIST)
    # EEPROM = Tool.i2c_read(device = 'iic_0', slv_addr = 0x50, reg_ptr = [0x02], byte_len = 4)
    # print('CUR Register Data = ',EEPROM)
    
    End_Time = time.time()
    print('Power Sweep Measurement Time Elapsed: ' + str(round(End_Time-Start_Time, 2)) + ' seconds')
    print('Power Sweep Measurement Finish.\n')
    
def Power_Fixed_Measurement(config_num=31, Min=-20, Max=0, Step=1):
    Start_Time = time.time()
    print('Power Detector Measurement Start.')
    
    constant_map = {
                      "PA_MNCOMP": True,
                      "PA_EN": True, "PA_DSVCG": 1.9, "PA_DSVG": 0.375, # PA_DSVCG, PA_DSVG unit = V
                      "PD_EN": True, "PA_PSVCG": 1.9, "PA_PSVG": 0.375, # PA_PSVCG, PA_PSVG unit = V
                      "PD_VDDEN": True, "PD_OUTEN": True, "BIAS_VDDEN": True, "PD_GAIN": 0b01010,
                      "TEMP_OUTEN": False, "TEMP_GAIN": 6,
                      "PD_RFEN": True, "COUPLER_RFEN": False, "COUPLER_50EN": True,

                      "COMPENS_VDDEN": True, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0.040625, "COMPENS_PS": 0.040625,
                      "TEMP_VDDEN": False,
                      "TEST_CTRL": "High-Z"      
    }
    
    
    
    Config_map = {
                #   0:[1.9, 0.375, 1.9, 0.375, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   1:[1.9, 0.350, 1.9, 0.375, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   2:[1.9, 0.325, 1.9, 0.375, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   3:[1.9, 0.300, 1.9, 0.375, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   4:[1.9, 0.275, 1.9, 0.375, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   5:[1.9, 0.250, 1.9, 0.375, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                  0:{"PA_DSVG": 0.375, "PA_PSVG": 0.375},                  
                  1:{"PA_DSVG": 0.350, "PA_PSVG": 0.375},
                  2:{"PA_DSVG": 0.325, "PA_PSVG": 0.375},
                  3:{"PA_DSVG": 0.300, "PA_PSVG": 0.375},
                  4:{"PA_DSVG": 0.275, "PA_PSVG": 0.375},
                  5:{"PA_DSVG": 0.250, "PA_PSVG": 0.375},
                  
                #   6:[1.9, 0.375, 1.9, 0.350, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   7:[1.9, 0.350, 1.9, 0.350, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   8:[1.9, 0.325, 1.9, 0.350, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   9:[1.9, 0.300, 1.9, 0.350, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   10:[1.9, 0.275, 1.9, 0.350, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   11:[1.9, 0.250, 1.9, 0.350, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                  6:{"PA_DSVG": 0.375, "PA_PSVG": 0.350},
                  7:{"PA_DSVG": 0.350, "PA_PSVG": 0.350},
                  8:{"PA_DSVG": 0.325, "PA_PSVG": 0.350},
                  9:{"PA_DSVG": 0.300, "PA_PSVG": 0.350},
                  10:{"PA_DSVG": 0.275, "PA_PSVG": 0.350},
                  11:{"PA_DSVG": 0.250, "PA_PSVG": 0.350},
                  
                #   12:[1.9, 0.375, 1.9, 0.325, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   13:[1.9, 0.350, 1.9, 0.325, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   14:[1.9, 0.325, 1.9, 0.325, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   15:[1.9, 0.300, 1.9, 0.325, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   16:[1.9, 0.275, 1.9, 0.325, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   17:[1.9, 0.250, 1.9, 0.325, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                  12:{"PA_DSVG": 0.375, "PA_PSVG": 0.325},
                  13:{"PA_DSVG": 0.350, "PA_PSVG": 0.325},
                  14:{"PA_DSVG": 0.325, "PA_PSVG": 0.325},
                  15:{"PA_DSVG": 0.300, "PA_PSVG": 0.325},
                  16:{"PA_DSVG": 0.275, "PA_PSVG": 0.325},
                  17:{"PA_DSVG": 0.250, "PA_PSVG": 0.325},
                  
                #   18:[1.9, 0.375, 1.9, 0.300, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   19:[1.9, 0.350, 1.9, 0.300, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   20:[1.9, 0.325, 1.9, 0.300, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   21:[1.9, 0.300, 1.9, 0.300, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   22:[1.9, 0.275, 1.9, 0.300, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   23:[1.9, 0.250, 1.9, 0.300, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                  18:{"PA_DSVG": 0.375, "PA_PSVG": 0.300},
                  19:{"PA_DSVG": 0.350, "PA_PSVG": 0.300},
                  20:{"PA_DSVG": 0.325, "PA_PSVG": 0.300},
                  21:{"PA_DSVG": 0.300, "PA_PSVG": 0.300},
                  22:{"PA_DSVG": 0.275, "PA_PSVG": 0.300},
                  23:{"PA_DSVG": 0.250, "PA_PSVG": 0.300},
                  
                #   24:[1.9, 0.375, 1.9, 0.275, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   25:[1.9, 0.350, 1.9, 0.275, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   26:[1.9, 0.325, 1.9, 0.275, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   27:[1.9, 0.300, 1.9, 0.275, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   28:[1.9, 0.275, 1.9, 0.275, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                #   29:[1.9, 0.250, 1.9, 0.275, 0, True, True, True, "011", "01001", "6", "PD", "TERM"],
                  24:{"PA_DSVG": 0.375, "PA_PSVG": 0.275},
                  25:{"PA_DSVG": 0.350, "PA_PSVG": 0.275},
                  26:{"PA_DSVG": 0.325, "PA_PSVG": 0.275},
                  27:{"PA_DSVG": 0.300, "PA_PSVG": 0.275},
                  28:{"PA_DSVG": 0.275, "PA_PSVG": 0.275},
                  29:{"PA_DSVG": 0.250, "PA_PSVG": 0.275},
                  
                #   30:[1.9, 0.375, 1.9, 0.375, 0, True, False, False, "011", "01001", "6", "PD", "TERM"],
                  30:{
                        "PA_MNCOMP": True,
                        "PA_EN": False, "PA_DSVCG": 1.9, "PA_DSVG": 0.375, # PA_DSVCG, PA_DSVG unit = V
                        "PD_EN": False, "PA_PSVCG": 1.9, "PA_PSVG": 0.375, # PA_PSVCG, PA_PSVG unit = V
                        "PD_VDDEN": False, "PD_OUTEN": True, "BIAS_VDDEN": False, "PD_GAIN": 0b01010,
                        "TEMP_OUTEN": False, "TEMP_GAIN": 6,
                        "PD_RFEN": True, "COUPLER_RFEN": False, "COUPLER_50EN": True,

                        "COMPENS_VDDEN": True, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0.040625, "COMPENS_PS": 0.040625,
                        "TEMP_VDDEN": False,
                        "TEST_CTRL": "High-Z"
                    },
                 }    

    print("============================= POWER_SUPPLY_ON() =================================")
    time.sleep(0.5)
    RFPGA_Power_Down()
    
    Config_num = range(config_num)
    Input_num = range(Min,Max+1,Step)

    Config = np.zeros(np.size(Input_num))
    Input = np.zeros(np.size(Input_num))
    Gain  = np.zeros(np.size(Input_num))
    Pin   = np.zeros(np.size(Input_num))
    Pout  = np.zeros(np.size(Input_num))
    PD    = np.zeros(np.size(Input_num))
    
    RFPGA = RFPGADriver(debug=False)

    for i in Config_num:
        for j in Input_num:

            constant_map.update(Config_map[i])
            x = RFPGA.Gen_Bistream(config=constant_map, MSB=True, to_hex=False)
            
            decimal_list = DECIMAL_LIST_GEN(x)
            #print("deciaml_list: ", decimal_list)
            RFPGA_Power_Config(decimal_list)

            Power_fixed(j)
            time.sleep(0.5)
            Num = (Input_num.index(j))
            #print(Num)

            Config[Num] = i
            Input[Num] = j

            VNA.write(":CALC1:MEAS1:MARK1:Y?")
            x = VNA.read()
            Temp =[float(d) for d in x.split(',')]
            Pin[Num] = Temp[0]

            VNA.write(":CALC1:MEAS2:MARK1:Y?")
            x = VNA.read()
            Temp =[float(d) for d in x.split(',')]
            Pout[Num] = Temp[0]

            PD[Num] = ADC_get()
            Power_unfixed()

        df = pd.DataFrame()
        df['Config'] = Config
        df['Input (dBm)'] = Input
        df['Gain (dB)'] = Pout-Pin
        df['Pin (dBm)'] = Pin
        df['Pout (dBm)'] = Pout
        df['PD (mV)'] = PD
        if(i == 0):
            poly_coefficient = Regression(Pout,PD)
            Poly = np.poly1d(poly_coefficient)
        else:
            for j in len(Input_num):
                Pout_w = 10 ** (Pout[j]/10)
                P_gald = Poly(PD[j])
                if((Pout_w-P_gald)/P_gald > 0.05):
                    print("[Error] Power regration isn't fit to config num", i)
        if not os.path.isdir(path+'\Fixed_Power'):
               os.makedirs(path+'\Fixed_Power')

        savefile_filename = path+'\Fixed_Power\Fixed_Power_Config'+str(i)+'.csv'    
        df.to_csv(savefile_filename)

    End_Time = time.time()
    print('Power Detector Measurement Time Elapsed: ' + str(round(End_Time-Start_Time, 2)) + ' seconds')
    print('Power Detector Measurement Finish.\n')
    
def Power_Coupler_Measurement(config_num=21, Min=-20, Max=0):
    Start_Time = time.time()
    print('Power_Coupler Measurement Start.')
    
    constant_map = {
                      "PA_MNCOMP": True,
                      "PA_EN": True, "PA_DSVCG": 1.9, "PA_DSVG": 0.375, # PA_DSVCG, PA_DSVG unit = V
                      "PD_EN": True, "PA_PSVCG": 1.9, "PA_PSVG": 0.375, # PA_PSVCG, PA_PSVG unit = V
                      "PD_VDDEN": False, "PD_OUTEN": False, "BIAS_VDDEN": True, "PD_GAIN": 0b01010,
                      "TEMP_OUTEN": False, "TEMP_GAIN": 6,
                      "PD_RFEN": False, "COUPLER_RFEN": True, "COUPLER_50EN": False,

                      "COMPENS_VDDEN": True, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0.040625, "COMPENS_PS": 0.040625,
                      "TEMP_VDDEN": False,
                      "TEST_CTRL": "High-Z"      
    }
    
    Config_map = {
                  # 0:[1.9, 0.375, 1.9, 0.375, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 1:[1.9, 0.350, 1.9, 0.375, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 2:[1.9, 0.325, 1.9, 0.375, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 3:[1.9, 0.300, 1.9, 0.375, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 4:[1.9, 0.275, 1.9, 0.375, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 5:[1.9, 0.250, 1.9, 0.375, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  0:{"PA_DSVG": 0.375, "PA_PSVG": 0.375},                  
                  1:{"PA_DSVG": 0.350, "PA_PSVG": 0.375},
                  2:{"PA_DSVG": 0.325, "PA_PSVG": 0.375},
                  3:{"PA_DSVG": 0.300, "PA_PSVG": 0.375},
                  4:{"PA_DSVG": 0.275, "PA_PSVG": 0.375},
                  5:{"PA_DSVG": 0.250, "PA_PSVG": 0.375},
                                 
                  # 6:[1.9, 0.375, 1.9, 0.350, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 7:[1.9, 0.350, 1.9, 0.350, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 8:[1.9, 0.325, 1.9, 0.350, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 9:[1.9, 0.300, 1.9, 0.350, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 10:[1.9, 0.275, 1.9, 0.350, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 11:[1.9, 0.250, 1.9, 0.350, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  6:{"PA_DSVG": 0.375, "PA_PSVG": 0.350},
                  7:{"PA_DSVG": 0.350, "PA_PSVG": 0.350},
                  8:{"PA_DSVG": 0.325, "PA_PSVG": 0.350},
                  9:{"PA_DSVG": 0.300, "PA_PSVG": 0.350},
                  10:{"PA_DSVG": 0.275, "PA_PSVG": 0.350},
                  11:{"PA_DSVG": 0.250, "PA_PSVG": 0.350},
                  
                  # 12:[1.9, 0.375, 1.9, 0.325, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 13:[1.9, 0.350, 1.9, 0.325, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 14:[1.9, 0.325, 1.9, 0.325, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 15:[1.9, 0.300, 1.9, 0.325, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 16:[1.9, 0.275, 1.9, 0.325, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 17:[1.9, 0.250, 1.9, 0.325, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  12:{"PA_DSVG": 0.375, "PA_PSVG": 0.325},
                  13:{"PA_DSVG": 0.350, "PA_PSVG": 0.325},
                  14:{"PA_DSVG": 0.325, "PA_PSVG": 0.325},
                  15:{"PA_DSVG": 0.300, "PA_PSVG": 0.325},
                  16:{"PA_DSVG": 0.275, "PA_PSVG": 0.325},
                  17:{"PA_DSVG": 0.250, "PA_PSVG": 0.325},
                  
                  # 18:[1.9, 0.375, 1.9, 0.300, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 19:[1.9, 0.350, 1.9, 0.300, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  # 20:[1.9, 0.325, 1.9, 0.300, 0, True, True, False, "011", "01001", "6", "HIGH-Z", "CAL"],
                  18:{"PA_DSVG": 0.375, "PA_PSVG": 0.300},
                  19:{"PA_DSVG": 0.350, "PA_PSVG": 0.300},
                  20:{"PA_DSVG": 0.325, "PA_PSVG": 0.300},
                 }

    print("============================= POWER_SUPPLY_ON() =================================")
    time.sleep(0.5)
    RFPGA_Power_Down()

    Config_num = range(config_num)
    
    RFPGA = RFPGADriver(debug=False)

    for i in Config_num:

        constant_map.update(Config_map[i])
        x = RFPGA.Gen_Bistream(config=constant_map, MSB=True, to_hex=False)
        
        decimal_list = DECIMAL_LIST_GEN(x)
        #print("deciaml_list: ", decimal_list)
        RFPGA_Power_Config(decimal_list)
        ChangeMeasurement('R_1','D_1','S41')
        Power_sweep(Min,Max,101,1E3)
        Output = "Coupler"
        time.sleep(1)
        CSV_get_Power_sweep_select(i,path,Output)
        ChangeMeasurement('R_1','B_1','S21')
        Power_sweep(Min,Max,101,1E3)
        Output = "RFout"
        time.sleep(1)
        CSV_get_Power_sweep_select(i,path,Output)

    End_Time = time.time()
    print('Time Elapsed: ' + str(round(End_Time-Start_Time, 2)) + ' seconds')
    print('Power_Coupler Measurement Finish.\n')


# Main

In [19]:
path = "./Measurement_Data"
if not os.path.isdir(path):
    os.makedirs(path)

uuid = check()
path = "./Measurement_Data/PGA_"+uuid+f"_{time.time()}"

try:
    Temp_Measurement()
    Power_Sweep_Measurement(config_num=31)
    Power_Fixed_Measurement(config_num=1)
    Power_Coupler_Measurement(config_num=1)
finally:
    RFPGA_Power_Down()
    print("============================= POWER_SUPPLY_OFF() =================================")

============================= POWER_SUPPLY_ON() =================================
uuid =  6060
[STATUS] Config Error
============================= POWER_SUPPLY_OFF() =================================
============================= POWER_SUPPLY_OFF() =================================


SystemExit: 0

c:\Users\Allen Li\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
text = "loading"
for i in range(5):
    text += "." * i
    print(text, "\r")


In [ ]:
try:
    Temp_Measurement()
finally:
    RFPGA_Power_Down()
    print("============================= POWER_SUPPLY_OFF() =================================")

In [ ]:
try:
    Power_Sweep_Measurement(config_num=31)
finally:
    RFPGA_Power_Down()
    print("============================= POWER_SUPPLY_OFF() =================================")

In [ ]:
try:
    Power_Fixed_Measurement(config_num=31)
finally:
    RFPGA_Power_Down()
    print("============================= POWER_SUPPLY_OFF() =================================")

In [ ]:
try:
    Power_Coupler_Measurement(config_num=21)
finally:
    RFPGA_Power_Down()
    print("============================= POWER_SUPPLY_OFF() =================================")

In [21]:
print("============================= POWER_SUPPLY_OFF() =================================")

In [98]:
def check2():
    print("============================= POWER_SUPPLY_ON() =================================")
    # POWER_SUPPLY_ON()
    time.sleep(0.5)

    RFPGA_Power_Down()
    
    # initialization
    RFPGA = RFPGADriver(debug=False)
    
    # check mode configuration
    check_mode_config = {
        "PA_MNCOMP": True,
        "PA_EN": True, "PA_DSVCG": 1.9, "PA_DSVG": 0.250, # PA_DSVCG, PA_DSVG unit = V
        "PD_EN": True, "PA_PSVCG": 1.9, "PA_PSVG": 0.275, # PA_PSVCG, PA_PSVG unit = V
        "PD_VDDEN": False, "PD_OUTEN": False, "BIAS_VDDEN": True, "PD_GAIN": 0b10100,
        "TEMP_OUTEN": False, "TEMP_GAIN": 6,
        "PD_RFEN": False, "COUPLER_RFEN": False, "COUPLER_50EN": True,
        
        "COMPENS_VDDEN": False, "COMPENS_CPTAT": "CTAT", "COMPENS_DS": 0, "COMPENS_PS": 0,
        "TEMP_VDDEN": False,
        "TEST_CTRL": "High-Z"        
    }
    
    x = RFPGA.Gen_Bistream(config=check_mode_config, MSB=True, to_hex=False)
    
    # =======================================================================================
    
    decimal_list = DECIMAL_LIST_GEN(x)
    RFPGA_Power_Config(decimal_list)

    time.sleep(0.5)
    
    # I2V5 = meas_supply1()[1]
    # I3V3 = meas_supply2()[1]
    # print('I2V5 = %.3f A' %I2V5,'I3V3 = %.3f A' %I3V3)

    readResult = RFPGA_Config_Read(decimal_list)
    # RFPGA_Power_Down()
    time.sleep(0.5)
    print("============================= POWER_SUPPLY_OFF() =================================")
    # POWER_SUPPLY_OFF()
        
    return readResult


In [99]:
print(check2())

============================= POWER_SUPPLY_ON() =================================
============================= POWER_SUPPLY_OFF() =================================
[array([[  0, 255, 255, 255],
       [  0, 255, 255, 255],
       [ 16, 255, 255, 255],
       [144, 255, 255, 255],
       [179, 255, 255, 255],
       [ 67, 255, 255, 255],
       [ 58, 255, 255, 255],
       [134, 255, 255, 255],
       [125, 255, 255, 255]], dtype=uint8)]


In [ ]:
POWER_SUPPLY_ON()

In [82]:
POWER_SUPPLY_OFF()

In [58]:
result = [0,0,16,145,67,195,39,135,125]
result_bin = ""
for i in result:
    result_bin += f'{i:08b}'
    result_bin += " "
    
gainEN_MSB = "10111110 11100001 11111100 11000011 11111010 10001001 00001100 10100101 00000000"[::-1]
    
print("normal:  10111110 11100001 11111100 11000011 11001010 10001001 00001100 10100101 00000000")
print("gainEN:  10111110 11100001 11111100 11000011 11111010 10001001 00001100 10100101 00000000")
print("gainEN_MSB:", gainEN_MSB)
print("config:", result_bin[::-1])

normal:  10111110 11100001 11111100 11000011 11001010 10001001 00001100 10100101 00000000
gainEN:  10111110 11100001 11111100 11000011 11111010 10001001 00001100 10100101 00000000
gainEN_MSB: 00000000 10100101 00110000 10010001 01011111 11000011 00111111 10000111 01111101
config:  10111110 11100001 11100100 11000011 11000010 10001001 00001000 00000000 00000000


In [ ]:
def RFPGA_Power_Config(Config):
    ## Config Power
    Tool.SPI_init(device = 'SPI_0', SPI_CSB_IDLE_STATE = 0, SPI_CSB_MASK = 0, SPI_CPOL = 0, SPI_CPHA = 0, SPI_SDIO_mode = 0)  
    w_data = [[Config, [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], [0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0]]]
    Tool.SPI_program(device = ['SPI_0'], w_data = w_data, r_bytes = [0])
    
POWER_SUPPLY_ON()


In [97]:
RFPGA_Power_Down()

In [ ]:
# 實際SPI寫入的函式

# power down mode
RFPGA_Power_Down()

# 寫入
RFPGA_Power_Config(Config)

# 讀取
RFPGA_Config_Read(Config)

In [ ]:
# 得到 config 的方法:

# 1. 在 RFPGADriver_V4.py 的 Gen_Bistream() 會先產生一組字串，例如 "10111110 11100001"
# 2. 利用 
DECIMAL_LIST_GEN()
#    把字串轉成十進位的list，例如 [190, 225]

# 得到的list就是我們要的config